In [1]:
from pyspark.sql import SparkSession
import numpy as np

In [2]:
spark = SparkSession.builder.appName("xG3").getOrCreate()

your 131072x1 screen size is bogus. expect trouble
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/23 23:47:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
events = spark.read.csv('Data/events.csv', header=True, inferSchema=True,sep=';')

In [4]:
import pp_events as pp

In [5]:
events_shot = pp.preprocessing(events,spark)

Data loaded
Spatial data calculated
Preferred foot calculated
Goal column created


Number of players inside the area calculated
Boolean data converted to integer


In [6]:
events_shot.columns

['id',
 'player_id',
 'shot_location_x',
 'shot_location_y',
 'distance_to_goal',
 'shot_angle',
 'preferred_foot_shot',
 'shot_body_part',
 'shot_technique',
 'shot_type',
 'play_pattern',
 'under_pressure',
 'shot_aerial_won',
 'shot_first_time',
 'shot_one_on_one',
 'shot_open_goal',
 'shot_follows_dribble',
 'players_inside_area',
 'shot_statsbomb_xg',
 'shot_outcome',
 'goal']

In [7]:
events_shot.show()

24/12/23 23:49:05 WARN TaskSetManager: Stage 5 contains a task of very large size (1681 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+---------+---------------+---------------+----------------+----------+-------------------+--------------+--------------+---------+--------------+--------------+---------------+---------------+---------------+--------------+--------------------+-------------------+-----------------+------------+----+
|                  id|player_id|shot_location_x|shot_location_y|distance_to_goal|shot_angle|preferred_foot_shot|shot_body_part|shot_technique|shot_type|  play_pattern|under_pressure|shot_aerial_won|shot_first_time|shot_one_on_one|shot_open_goal|shot_follows_dribble|players_inside_area|shot_statsbomb_xg|shot_outcome|goal|
+--------------------+---------+---------------+---------------+----------------+----------+-------------------+--------------+--------------+---------+--------------+--------------+---------------+---------------+---------------+--------------+--------------------+-------------------+-----------------+------------+----+
|350a49bc-6914-4b3...|  27886.0

In [16]:
# since the dataframe is final, we save it into a variable in memory
events_shot = events_shot.cache()

24/12/23 23:51:24 WARN CacheManager: Asked to cache already cached data.


In [19]:
events_shot.printSchema()

root
 |-- id: string (nullable = true)
 |-- player_id: double (nullable = false)
 |-- shot_location_x: float (nullable = false)
 |-- shot_location_y: float (nullable = false)
 |-- distance_to_goal: double (nullable = false)
 |-- shot_angle: float (nullable = false)
 |-- preferred_foot_shot: integer (nullable = true)
 |-- shot_body_part: string (nullable = true)
 |-- shot_technique: string (nullable = true)
 |-- shot_type: string (nullable = true)
 |-- play_pattern: string (nullable = true)
 |-- under_pressure: integer (nullable = true)
 |-- shot_aerial_won: integer (nullable = true)
 |-- shot_first_time: integer (nullable = true)
 |-- shot_one_on_one: integer (nullable = true)
 |-- shot_open_goal: integer (nullable = true)
 |-- shot_follows_dribble: integer (nullable = true)
 |-- players_inside_area: integer (nullable = true)
 |-- shot_statsbomb_xg: double (nullable = false)
 |-- shot_outcome: string (nullable = true)
 |-- goal: integer (nullable = true)



In [21]:
#features = ['distance_to_goal', 'shot_angle', 'preferred_foot_shot', 'under_pressure']
features = [#'shot_location_x', 'shot_location_y',
            'distance_to_goal', 'shot_angle', 'preferred_foot_shot', 'under_pressure',
            'shot_aerial_won','shot_first_time','shot_one_on_one','shot_open_goal','shot_follows_dribble','players_inside_area']
target = ['goal']

In [22]:
from pyspark.ml.feature import VectorAssembler

# Define the assembler
feature_assembler = VectorAssembler(inputCols=features, outputCol="features_vector")

# Transform the dataset
assembled_data = feature_assembler.transform(events_shot)
train_data, test_data = assembled_data.randomSplit([0.8, 0.2], seed=42)

In [24]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline


# Define the logistic regression model
log_reg = LogisticRegression(featuresCol="features_vector", labelCol="goal", maxIter=10)
# Create the pipeline
pipeline = Pipeline(stages=[log_reg])

In [26]:
xg_model = pipeline.fit(train_data)

24/12/23 23:53:48 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


In [27]:
# Make predictions
predictions = xg_model.transform(test_data)

# Evaluate using BinaryClassificationEvaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(labelCol="goal", rawPredictionCol="rawPrediction")
roc_auc = evaluator.evaluate(predictions)
print(f"ROC-AUC: {roc_auc}")

ROC-AUC: 0.8008854126467988


In [28]:
from pyspark.sql.functions import col, udf, format_number, abs
from pyspark.sql.types import DoubleType

# Define a function to extract the probability of the goal (class 1)
def extract_goal_probability(probability):
    return float(probability[1])

# Register the function as a UDF
extract_goal_probability_udf = udf(extract_goal_probability, DoubleType())

# Create a new column with the goal probability
predictions_with_goal_prob = predictions.withColumn("goal_probability", extract_goal_probability_udf(col("probability")))

# Format the goal_probability to remove scientific notation
predictions_with_goal_prob = predictions_with_goal_prob.withColumn("goal_probability", format_number(col("goal_probability"), 10))

# Add a new column showing the difference between shot_statsbomb_xg and goal_probability
predictions_with_goal_prob = predictions_with_goal_prob.\
    withColumn("difference", format_number(col("shot_statsbomb_xg") - col("goal_probability"),10))

predictions_with_goal_prob = predictions_with_goal_prob.\
    withColumn("difference_abs", abs(col('difference')))

# Show the results
predictions_with_goal_prob.select("shot_statsbomb_xg", "goal_probability", "difference",'difference_abs').show(50,False)

+-----------------+----------------+-------------+--------------+
|shot_statsbomb_xg|goal_probability|difference   |difference_abs|
+-----------------+----------------+-------------+--------------+
|0.04334119       |0.0518483519    |-0.0085071619|0.0085071619  |
|0.05638644       |0.0753202060    |-0.0189337660|0.018933766   |
|0.044450935      |0.1425743673    |-0.0981234323|0.0981234323  |
|0.03511139       |0.0437537090    |-0.0086423190|0.008642319   |
|0.049519155      |0.0892330998    |-0.0397139448|0.0397139448  |
|0.022752915      |0.0357739073    |-0.0130209923|0.0130209923  |
|0.0752324        |0.1100162500    |-0.0347838500|0.03478385    |
|0.11348543       |0.2707885197    |-0.1573030897|0.1573030897  |
|0.026074365      |0.0125890938    |0.0134852712 |0.0134852712  |
|0.026830109      |0.0401478655    |-0.0133177565|0.0133177565  |
|0.086427316      |0.0759482464    |0.0104790696 |0.0104790696  |
|0.05394276       |0.1220326285    |-0.0680898685|0.0680898685  |
|0.0138394

In [29]:
# average of the difference
from pyspark.sql.functions import avg

avg_diff = predictions_with_goal_prob.select(avg(col("difference"))).collect()[0][0]
print(f"Average difference: {avg_diff}")

abs_avg_diff = predictions_with_goal_prob.select(avg(col("difference_abs"))).collect()[0][0]
print(f"Abs average difference: {abs_avg_diff}")

Average difference: -0.0031711406162004386


Abs average difference: 0.05225033141487065


In [30]:
# round the goal_probability to 0 or 1, same for the shot_statsbomb_xg and compare with goal to check the accuracy of the model
# vs the accuracy of the shot_statsbomb_xg model

from pyspark.sql.functions import round

predictions_with_goal_prob = predictions_with_goal_prob.withColumn("goal_probability_rounded", round(col("goal_probability")))\
    .withColumn("shot_statsbomb_xg_rounded", round(col("shot_statsbomb_xg")))

predictions_with_goal_prob.select("goal", "goal_probability_rounded", "shot_statsbomb_xg_rounded").show()

+----+------------------------+-------------------------+
|goal|goal_probability_rounded|shot_statsbomb_xg_rounded|
+----+------------------------+-------------------------+
|   0|                     0.0|                      0.0|
|   0|                     0.0|                      0.0|
|   0|                     0.0|                      0.0|
|   0|                     0.0|                      0.0|
|   0|                     0.0|                      0.0|
|   0|                     0.0|                      0.0|
|   1|                     0.0|                      0.0|
|   0|                     0.0|                      0.0|
|   0|                     0.0|                      0.0|
|   0|                     0.0|                      0.0|
|   0|                     0.0|                      0.0|
|   0|                     0.0|                      0.0|
|   0|                     0.0|                      0.0|
|   0|                     0.0|                      0.0|
|   0|        

In [31]:
from pyspark.sql.functions import col, when

# Check if the rounded probability matches the actual goal for your model
predictions_with_accuracy = predictions_with_goal_prob.withColumn(
    "model_correct", when(col("goal") == col("goal_probability_rounded"), 1).otherwise(0)
).withColumn(
    "statsbomb_correct", when(col("goal") == col("shot_statsbomb_xg_rounded"), 1).otherwise(0)
)

In [32]:
# Calculate accuracy
accuracy_model = predictions_with_accuracy.selectExpr("avg(model_correct) as model_accuracy").first()["model_accuracy"]
accuracy_statsbomb = predictions_with_accuracy.selectExpr("avg(statsbomb_correct) as statsbomb_accuracy").first()["statsbomb_accuracy"]

print(f"Model Accuracy: {accuracy_model * 100:.2f}%")
print(f"StatsBomb xG Accuracy: {accuracy_statsbomb * 100:.2f}%")

Model Accuracy: 89.87%
StatsBomb xG Accuracy: 90.90%


In [33]:
# show each feature with its coefficient
coefficients = xg_model.stages[-1].coefficients
features_coefficients = list(zip(features, coefficients))

for feature, coefficient in features_coefficients:
    print(f"{feature}: {coefficient}")

distance_to_goal: -0.0996929612307214
shot_angle: 0.022848632580391395
preferred_foot_shot: 0.8673798572513546
under_pressure: -0.20892695425070718
shot_aerial_won: -0.95174524161054
shot_first_time: -0.2685325219729587
shot_one_on_one: 0.28640098073007514
shot_open_goal: 0.8822105198942372
shot_follows_dribble: 0.9049464738620332
players_inside_area: -0.2899164157444605


In [34]:
# Testing matching of models, is out model as good as SB_xg?
predictions_with_accuracy = predictions_with_goal_prob.withColumn(
    "model_to_sbxg", when(col("goal_probability_rounded") == col("shot_statsbomb_xg_rounded"), 1).otherwise(0)
)

In [35]:
xg_accuracy_model = predictions_with_accuracy.selectExpr("avg(model_to_sbxg) as model_accuracy").first()["model_accuracy"]

print(f"Models matching percentage: {xg_accuracy_model * 100:.2f}%")

Models matching percentage: 97.04%


In [38]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import col

# Create an evaluator for accuracy, precision, recall, and F1-score
accuracy_evaluator = MulticlassClassificationEvaluator(labelCol="goal", predictionCol="prediction", metricName="accuracy")
precision_evaluator = MulticlassClassificationEvaluator(labelCol="goal", predictionCol="prediction", metricName="weightedPrecision")
recall_evaluator = MulticlassClassificationEvaluator(labelCol="goal", predictionCol="prediction", metricName="weightedRecall")
f1_evaluator = MulticlassClassificationEvaluator(labelCol="goal", predictionCol="prediction", metricName="f1")

# Get predictions
predictions = best_model.transform(test_data)

# Compute accuracy for the whole dataset
accuracy = accuracy_evaluator.evaluate(predictions)
print(f"Overall Accuracy: {accuracy:.4f}")

# Compute precision, recall, and F1-score for the whole dataset
precision = precision_evaluator.evaluate(predictions)
recall = recall_evaluator.evaluate(predictions)
f1_score = f1_evaluator.evaluate(predictions)

print(f"Overall Precision: {precision:.4f}")
print(f"Overall Recall: {recall:.4f}")
print(f"Overall F1-Score: {f1_score:.4f}")

# Generate per-class metrics (you can use confusion matrix to get counts per class)
from pyspark.ml.linalg import DenseVector
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

# Extract prediction and label columns
pred_and_labels = predictions.select("prediction", "goal")

# Calculate confusion matrix
confusion_matrix = pred_and_labels.groupby("goal", "prediction").count().collect()

# Convert confusion matrix into a dictionary for easy reading
confusion_dict = {}
for row in confusion_matrix:
    goal_class = row["goal"]
    predicted_class = row["prediction"]
    confusion_dict[(goal_class, predicted_class)] = row["count"]

# Display confusion matrix for each class
print("Confusion Matrix:")
for goal_class in set(pred_and_labels.select("goal").rdd.flatMap(lambda x: x).collect()):
    for predicted_class in set(pred_and_labels.select("prediction").rdd.flatMap(lambda x: x).collect()):
        print(f"Goal: {goal_class}, Predicted: {predicted_class}, Count: {confusion_dict.get((goal_class, predicted_class), 0)}")

# Calculate per-class accuracy (accuracy = TP / (TP + FP + FN + TN))
# You can compute this based on the confusion matrix for each class.


Overall Accuracy: 0.8989
Overall Precision: 0.8776
Overall Recall: 0.8989
Overall F1-Score: 0.8608
Confusion Matrix:


Goal: 0, Predicted: 0.0, Count: 10991
Goal: 0, Predicted: 1.0, Count: 46


Goal: 1, Predicted: 0.0, Count: 1201
Goal: 1, Predicted: 1.0, Count: 95
